In [12]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform
from tqdm import tqdm

In [42]:
node_df = pd.read_csv("node_uptime_with_locations.csv")
norway_pop_gdf = gpd.read_file("Population Data/Befolkning_0000_Norge_25833_BefolkningsstatistikkRutenett1km2023_GML.gml", driver="GML")

In [3]:
# Convert lat and lon to coordinates used by population Dataframe
P4326 = Proj(init='epsg:4326') # Input coordinate system EPSG:4326 WGS 84 
P25833 = Proj(init='epsg:25833') # Output coordinate system: EPSG:25833 ETRS89 / UTM zone 33N

/home/nic/School/bachelor/norway-population-density/venv-npd/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/nic/School/bachelor/norway-population-density/venv-npd/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [4]:
def polygon_to_center(polygon):
    coords = list(polygon.exterior.coords)
    top_left = coords[0]
    bottom_right = coords[2]

    mid = ((top_left[0] + bottom_right[0]) / 2, (top_left[1] + bottom_right[1]) / 2)
    return mid

# Transforms coordinate in EPSG:25833 ETRS89 / UTM zone 33N to EPSG:4326 WGS 84 (regular lat and lon coord system)
#def coords_proj(polygon_coords):
#    x, y = polygon_coords
#    return transform(P25833, P4326, x, y)
#
#
#def polygon_to_pos(polygon):
#    return coords_proj(polygon_to_center(polygon))

In [5]:
#polygons_and_pop = zip(norway_pop_gdf["geometry"], norway_pop_gdf["popTot"])
length = 55208
from dataclasses import dataclass

@dataclass
class PopSquare:
    x: float
    y: float
    pop: float


pop_squares = []

for i, (polygon, pop) in enumerate(zip(norway_pop_gdf["geometry"], norway_pop_gdf["popTot"])):
    if i % 5_000 == 0:
        print(i)
    #lon, lat = polygon_to_pos(polygon)
    x, y = polygon_to_center(polygon)
    pop_squares.append(PopSquare(x, y, pop))  

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [6]:
print(pop_squares[0])

PopSquare(x=610499.999999851, y=7596499.999998342, pop=33)


In [7]:
import math
import heapq


def proj_4326_to_25833(lon, lat):
    return transform(P4326, P25833, lon, lat)

def proj_25833_to_4326(x, y):
    return transform(P25833, P4326, x, y)

# O(n)
def find_closest_squares(lon, lat, n_squares=1):
    queue = []
    x, y = proj_4326_to_25833(lon, lat)
    # (square, Euclidean Distance from entries center)
    closest = [(None, math.inf) for _ in range(n_squares)]
    
    for square in pop_squares:
        square_x, square_y = square.x, square.y
        distance = math.sqrt(((square_x - x) ** 2) + ((square_y - y) ** 2))
        heapq.heappush(queue, (-distance, square))
        if len(queue) > n_squares:
            heapq.heappop(queue)

    return [square for _, square in queue]

In [11]:
squares = find_closest_squares(8.2235251, 60.5411714, n_squares=4)
square = squares[3]
lon, lat = proj_25833_to_4326(square.x, square.y)
print(lon, lat, square.pop)

8.221209891467721 60.53802468715438 430


/tmp/ipykernel_1741/3620030080.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return transform(P4326, P25833, lon, lat)
/tmp/ipykernel_1741/3620030080.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return transform(P25833, P4326, x, y)


In [19]:
populations = []
for lon, lat in zip(node_df["loc_lon"], node_df["loc_lat"]):
    closest_squares = find_closest_squares(lon, lat, n_squares=4)
    populations.append(sum(square.pop for square in closest_squares))

/tmp/ipykernel_1741/3620030080.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return transform(P4326, P25833, lon, lat)


In [47]:
from enum import Enum

class PopulationCategory(Enum):
    HIGH = 1
    MEDIUM = 2
    LOW = 3

PopulationCategory = Enum('PopulationCategory', ['HIGH', 'MEDIUM', 'LOW'])
# https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9663160
HIGH = 15_000
MEDIUM = 10_000

population_in_categories: list[PopulationCategory] = []

for population in populations:
    if population > HIGH:
        population_in_categories.append(PopulationCategory.HIGH.value)
    elif population > MEDIUM:
        population_in_categories.append(PopulationCategory.MEDIUM.value)
    else:
        population_in_categories.append(PopulationCategory.LOW.value)

In [48]:
node_df = node_df.assign(population=populations)
node_df = node_df.assign(population_category=population_in_categories)

In [52]:
node_df.describe()
node_df.to_csv("updated_node_uptime_with_locations.csv", index=False)